In [63]:
#Only run if needed
#!pip install pyarrow

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, year, month
from pyspark.sql.functions import col, rand
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import BucketedRandomProjectionLSH
import pyspark.sql.functions as F
import random
import os
import shutil
from pyspark.sql.functions import lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import lower, regexp_replace
from pyspark.sql.functions import col, when
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, regexp_extract
spark = SparkSession.builder \
    .appName("SplitByYearMonth") \
    .getOrCreate()

In [36]:
df_new = spark.read.csv(
    "file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv",
    header=True,
    inferSchema=True,
    ignoreLeadingWhiteSpace=True
)

In [37]:
df_new.show(5)

+------+--------------------+-------------+-------------------+---------+-------+----------+--------------------+--------------+----+----+----+----+----+
|origen|                date|     username|      user_fullname|n_replies|n_likes|n_retweets|                text|tweet_language| _c9|_c10|_c11|_c12|_c13|
+------+--------------------+-------------+-------------------+---------+-------+----------+--------------------+--------------+----+----+----+----+----+
|   df1|2019-05-27 11:49:...|    bitcointe|          Bitcointe|        0|      0|         0|Cardano: Digitize...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...|    3eyedbran|Bran - 3 Eyed Raven|        0|      2|         1|Another Test twee...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...|DetroitCrypto|        J. Scardina|        0|      0|         0|Current Crypto Pr...|            en|NULL|NULL|NULL|NULL|NULL|
|   df1|2019-05-27 11:49:...| mmursaleen72| Muhammad Mursaleen|        0|   

25/12/04 14:21:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv


In [38]:
df_setiment = df_new

In [39]:
df_setiment = df_setiment.withColumn(
    "clean_text",
    lower(regexp_replace("text", "[^a-zA-Z0-9 ]", ""))
)

In [40]:
#Making a label column

In [42]:
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql.functions import pandas_udf
import pandas as pd

sia = SentimentIntensityAnalyzer()

# Define pandas UDF with type hints (preferred)
@pandas_udf("int")
def vader_sentiment(text_series: pd.Series) -> pd.Series:
    def get_label(text):
        score = sia.polarity_scores(text)["compound"]
        if score > 0.05:
            return 1   # positive
        elif score < -0.05:
            return 0   # negative
        else:
            return 2   # neutral
    return text_series.apply(get_label)

# Apply to your DataFrame
df_setiment = df_setiment.withColumn("label", vader_sentiment("clean_text"))


In [45]:
from pyspark.sql.functions import col

#Filter nulls
df_setiment = df_setiment.filter(col("clean_text").isNotNull())

# Count how many rows are assigned to each label
label_counts = df_setiment.groupBy("label").count()

# Show the results
label_counts.show()


/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


+-----+-----+
|label|count|
+-----+-----+
|    1|  148|
|    2|  115|
|    0|   43|
+-----+-----+



In [53]:
df_setiment = df_setiment.filter(col("clean_text").isNotNull())

In [54]:
train, test = df_setiment.randomSplit([0.8, 0.2], seed=1234)

In [55]:
#Pipeline

In [56]:
# Tokenize the tweets
tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Convert words to term frequency vectors
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)

# Compute TF-IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")


In [57]:
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

In [58]:
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])

In [59]:
train_data, test_data = df_setiment.randomSplit([0.8, 0.2], seed=42)

In [60]:
model = pipeline.fit(train_data)

25/12/04 14:26:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv
/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/storage/home/eml6069/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
25/12/04 14:26:32 WARN CSVHeaderChecker: CSV header does not conform to

In [61]:
predictions = model.transform(test_data)
predictions.select("clean_text", "label", "prediction").show(10)

+--------------------+-----+----------+
|          clean_text|label|prediction|
+--------------------+-----+----------+
|bitcoin satoshi c...|    2|       1.0|
|httpstcoip0ph8uzy...|    2|       1.0|
|markdow bitcoin i...|    2|       2.0|
| could not agree ...|    0|       2.0|
|never too late to...|    1|       1.0|
|retweet and tweet...|    1|       1.0|
|them listen up  i...|    2|       0.0|
|bitcoin at 8000 i...|    1|       2.0|
|no youre really n...|    0|       1.0|
|hot hot hot      ...|    2|       2.0|
+--------------------+-----+----------+
only showing top 10 rows


25/12/04 14:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv


In [62]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.40425531914893614


25/12/04 14:26:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, , , , , 
 Schema: origen, date, username, user_fullname, n_replies, n_likes, n_retweets, text, tweet_language, _c9, _c10, _c11, _c12, _c13
Expected: _c9 but found: 
CSV file: file:///storage/work/eml6069/DS410/DS410_Final/sorted_output/results.csv
